In [3]:
import numpy as np
import pandas as pd


data = pd.read_csv('./Laptop_price.csv')

storage_capacity_values = data['Storage_Capacity'].values
storage_capacity_values = np.array(storage_capacity_values)
storage_capacity_values = (storage_capacity_values - storage_capacity_values.mean()) / storage_capacity_values.std()

processor_speed_values = data['Processor_Speed'].values
processor_speed_values = np.array(processor_speed_values)
processor_speed_values = (processor_speed_values - processor_speed_values.mean()) / processor_speed_values.std()

screen_size_values = data['Screen_Size'].values
screen_size_values = np.array(screen_size_values)
screen_size_values = (screen_size_values - screen_size_values.mean()) / screen_size_values.std()

X = []

for i in range(len(storage_capacity_values)):
    X.append([storage_capacity_values[i], processor_speed_values[i], screen_size_values[i]])
X = np.array(X)
X_train = X[:700]
X_test = X[-300:]


In [4]:
price_values = data['Price'].values

# определяем количество групп и диапазон
num_groups = 3
max_val = 35000
min_val = 7000

y = []

price_bins = np.linspace(min_val, max_val, num_groups + 1)


for price in price_values:
    group_index = np.argmax(price_bins > price) - 1
    group_vector = [0] * num_groups
    group_vector[group_index] = 1
    y.append(group_vector)

y = np.array(y)

# Удаляем столбцы, где все элементы равны 0
#y = y[:, ~np.all(y == 0, axis=0)]


y_train = y[:700]
y_test = y[-300:]

for x_val, y_val in zip(X_train, y_train):
    print(x_val, y_val)

[-0.23166366  1.47612831 -1.68421927] [0 1 0]
[ 1.32603954  0.22178769 -1.61018863] [0 0 1]
[-1.04881944  0.6713098  -1.29251017] [1 0 0]
[-0.23166366  1.44325129 -1.04187661] [0 1 0]
[1.32603954 0.70749852 0.54784274] [0 0 1]
[-1.04881944 -1.18844285 -1.23917675] [1 0 0]
[-1.04881944  0.25991564 -1.37448904] [1 0 0]
[-1.04881944  0.36391286  0.89824018] [1 0 0]
[-0.23166366 -0.26015335  0.56863684] [0 1 0]
[-0.23166366  0.80732212 -0.26090133] [0 1 0]
[1.32603954 1.48381439 0.14021409] [0 0 1]
[-0.23166366  1.45374146 -0.69493579] [0 1 0]
[-0.23166366 -0.16886407 -0.09488818] [0 1 0]
[-1.04881944 -1.32277211 -0.00982604] [1 0 0]
[-1.04881944  1.65633317 -0.43011942] [1 0 0]
[-1.04881944  1.15750535  1.39742147] [1 0 0]
[-1.04881944 -1.28372316 -1.73628657] [1 0 0]
[-1.04881944  1.43758602  0.95965424] [1 0 0]
[-1.04881944  1.26345624 -1.35510483] [1 0 0]
[-1.04881944  0.06355276  1.01376276] [1 0 0]
[1.32603954 0.31114087 0.3745872 ] [0 0 1]
[ 1.32603954 -0.34603976  1.17468931] [0 0 

In [5]:
learning_rate = 0.5
epochs = 2000

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

def initialize_weights_and_biases(layer_sizes):
    np.random.seed(1)
    num_layers = len(layer_sizes)
    weights = []
    biases = []
    for i in range(1, num_layers):
        weights.append(np.random.uniform(size=(layer_sizes[i - 1], layer_sizes[i])))
        biases.append(np.random.uniform(size=(1, layer_sizes[i])))
    return weights, biases


def forward_propagation(X, weights, biases):
    num_layers = len(weights) + 1
    layer_outputs = [X]
    for i in range(num_layers - 1):
        layer_input = np.dot(layer_outputs[-1], weights[i]) + biases[i]
        layer_output = sigmoid(layer_input)
        layer_outputs.append(layer_output)
    return layer_outputs


def backward_propagation(X, y, layer_outputs, weights, biases, learning_rate):
    num_layers = len(weights)
    num_samples = X.shape[0]
    output_layer_output = layer_outputs[-1]
    output_error = y - output_layer_output
    output_delta = output_error * sigmoid_derivative(output_layer_output)
    for i in range(num_layers - 1, -1, -1):
        if i == num_layers - 1:
            layer_delta = output_delta
        else:
            layer_error = layer_delta.dot(weights[i + 1].T)
            layer_delta = layer_error * sigmoid_derivative(layer_outputs[i + 1])
        weights[i] += layer_outputs[i].T.dot(layer_delta) * learning_rate / num_samples
        biases[i] += np.sum(layer_delta, axis=0, keepdims=True) * learning_rate / num_samples
    return weights, biases


def train_network(X, y, layer_sizes, learning_rate, epochs):
    weights, biases = initialize_weights_and_biases(layer_sizes)
    for epoch in range(epochs):
        layer_outputs = forward_propagation(X, weights, biases)
        weights, biases = backward_propagation(X, y, layer_outputs, weights, biases, learning_rate)
    print("Training complete!")
    return weights, biases

layer_sizes = [X.shape[1], 10, y.shape[1]]

weights, biases = train_network(X_train, y_train, layer_sizes, learning_rate, epochs)

Training complete!


In [6]:
def predict(X, price_values, weights, biases):
    layer_outputs = forward_propagation(X, weights, biases)
    predictions = np.argmax(layer_outputs[-1], axis=1)
    length = (max_val - min_val) / num_groups
    start = predictions * length + min_val
    end = (predictions + 1) * length + min_val
    result = np.column_stack((price_values, start, end))
    return predictions, result

# Функция для вычисления Precision и Recall для каждой компоненты
def calculate_precision_recall(tps, fps, fns):
    precisions, recalls = [], []
    for tp, fp, fn in zip(tps, fps, fns):
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        precisions.append(precision)
        recalls.append(recall)
    return precisions, recalls

predictions, result = predict(X_test, price_values[-300:], weights, biases)

predictions_array = np.zeros((len(predictions), y.shape[1]), dtype=int)

for i, pred in enumerate(predictions):
    predictions_array[i, pred] = 1

# Функция для вычисления терминов для каждой компоненты
def calculate_terms(predicted, actual):
    tps, fps, tns, fns = [], [], [], []
    for i in range(predicted.shape[1]):
        tp = sum((predicted[:, i] == 1) & (actual[:, i] == 1))
        fp = sum((predicted[:, i] == 1) & (actual[:, i] == 0))
        tn = sum((predicted[:, i] == 0) & (actual[:, i] == 0))
        fn = sum((predicted[:, i] == 0) & (actual[:, i] == 1))
        tps.append(tp)
        fps.append(fp)
        tns.append(tn)
        fns.append(fn)
    return tps, fps, tns, fns


# Вызываем функцию для вычисления терминов
tps, fps, tns, fns = calculate_terms(predictions_array, y_test)
print(tps)
print(fps)
print(fns)


# Вызываем функцию для вычисления Precision и Recall
precisions, recalls = calculate_precision_recall(tps, fps, fns)

# Выводим результаты
for i in range(len(precisions)):
    print(f"Precision: {round(precisions[i], 2)}, Recall: {recalls[i]}")


[106, 94, 98]
[0, 2, 0]
[2, 0, 0]
Precision: 1.0, Recall: 0.9814814814814815
Precision: 0.98, Recall: 1.0
Precision: 1.0, Recall: 1.0
